In [ ]:
import pandas as pd
import numpy as np

import matplotlib.dates as mdates
import lowess
import plotly.graph_objects as go
import plotly.subplots as sp
import matplotlib.cm as cm
from prettytable import PrettyTable
import base64


import warnings
import plotly.express as px


pd.set_option('display.max_columns', None)


In [ ]:
# df_items = pd.read_csv('../ATSH/IFASIS_for_python_atsh.csv')
df_items = pd.read_csv('IFASIS_for_python_sit-matt.csv')
df_items.columns = ['dimension', 'short_items','items', 'Importance', 'Score', 'context', 'items_hannah']

In [ ]:
# df = pd.read_csv('merged_ifasis-8-21.csv')
# df
# # df.columns = ['dimension', 'short_items','items', 'Importance', 'Score', 'context', 'items_hannah']

In [ ]:
df = pd.read_csv('sitt-matt_ifasis.csv')
df


In [ ]:
df_temp = df.drop(columns = ['Unnamed: 0', 'program_id'])

In [ ]:
# df_temp = df.drop(columns = ['Q147','Unnamed: 1', 'Q144', 'Q142', 'Q143'])

In [ ]:
df_scores = df_temp.iloc[:, ::2]  # Starts from the first column and selects every other column
df_importances = df_temp.iloc[:, 1::2]  # Starts from the second column and selects every other column

In [ ]:
df_scores

In [ ]:
df_importances

In [ ]:
## TEMP, just for may 2024 id04
df_importances.loc[1] = df_importances.loc[1].fillna(2)
df_importances


In [ ]:
dfs = []
imp_map = {1: 'not important', 2: 'somewhat important', 3: 'very important'}

for idx in range(0, len(df_importances)):
    df_site = pd.DataFrame()
 
    df_site['dimension'] = df_items['dimension']
    df_site['context'] = df_items['context']
    df_site['items_hannah'] = df_items['items_hannah']
    
    df_site['Importance'] = df_importances.iloc[idx].values.astype(int)
    df_site['Score'] = df_scores.iloc[idx].values.astype(int)
    
    df_site['site_name'] = df['program_id'].iloc[idx]
    
    # Apply the dictionary using map()
    df_site['Importance_str']  = df_site['Importance'].map(imp_map)
    # Convert 'Category' column to an ordered categorical data type
    df_site['Importance_str'] = pd.Categorical(df_site['Importance_str'], categories=['not important', 'somewhat important', 'very important'], ordered=True)

    dfs.append(df_site)

In [ ]:
d_map = {'D1': 'External', 'D2': 'Internal', 'D3': 'Dynamicare Health (DCH)', 'D4': 'Patients'}


In [ ]:
# for df in dfs:
#     df_curr = df.copy()
#     df_curr['dimension_str'] = df_curr['dimension'].map(d_map)
#     df_curr = df_curr[['dimension_str', 'context', 'items_hannah',"Importance_str", "Score"]]
#     df_curr.columns = ['Dimension', 'Context', 'Item',"Importance", "Score"]
#     print(df['site_name'].iloc[1]) # change based on where the site name is
#         # Determine an appropriate height based on row count.
#     # 40 is for header, and 30 is an estimated height for each row. 
#     # You can adjust these values based on your preference.
#     height = 40 + len(df_curr)*40


#     # Create table
#     fig = go.Figure(data=[go.Table(
#         header=dict(values=list(df_curr.columns),
#                     fill_color='paleturquoise',
#                     align='left'),
#         cells=dict(values=[df_curr[col] for col in df_curr.columns],
#                    fill_color='lavender',
#                    align='left'), columnwidth=[0.5, 1, 1, 0.5, 0.25] ),
        
#     ],  layout=dict(height=height, width = 900))

#     fig.show()

#     fig.write_image("images_out/SITT-MATT_tables/" + df['site_name'].loc[0] + "_IFASIS_table.png")
    


In [ ]:
# Function to sort each group based on 'Value'
def custom_sort(group):
    return group.sort_values(by=['Importance', 'Score'], ascending = False)


In [ ]:
# Function to encode image to Base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return f"data:image/png;base64,{encoded_string}"


In [ ]:
def plot_df(df):
    # Apply the custom sorting function within each group
    sorted_df = df.groupby('dimension').apply(custom_sort).reset_index(drop=True)

    # Insert new rows as specified
    new_row_positions = [4, 17, 21]
    new_row_values = [' ', '', '  ']
    for position, value in zip(new_row_positions, new_row_values):
        new_row = {'items_hannah': value}
        sorted_df = sorted_df.loc[:position].append(new_row, ignore_index=True).append(sorted_df[position+1:], ignore_index=True)
    
    color_map = {'not important': '#e6e7e8', 'somewhat important': '#adadad', 'very important': '#535454'}   

    # Create scatter traces for each "Importance_str" value
    scatter_traces = []
    for importance_value in color_map.keys():
        scatter_trace = go.Scatter(
            x=sorted_df[sorted_df['Importance_str'] == importance_value]['Score'].iloc[::-1],
            y=sorted_df[sorted_df['Importance_str'] == importance_value]['items_hannah'].iloc[::-1],
            mode='markers',
            name=importance_value,  # Set the name for the legend
            marker=dict(color=color_map[importance_value]),
            showlegend=False
        )
        scatter_traces.append(scatter_trace)

    # Create the figure using all scatter traces
    fig = go.Figure(data=scatter_traces)

    fig.update_layout(
        title='<b>Factors Affecting the Implementation of MOUD</b>',
        title_font=dict(size=25),
        title_x=0.5,
        margin=dict(l=500, r=10, t=8, b=10, pad=8),

        font=dict(family="Century Gothic",size=20,color='black'),
        height=1200,
        width=1100,
        yaxis=dict(categoryorder='array',
            categoryarray=sorted_df['items_hannah'].iloc[::-1].values,
            autorange=False,
            range=[-1, len(sorted_df)]), xaxis=dict(range=[0, 6]),
        legend_title_text='',
        legend=dict(
            x=-0.9,
            y=-0.03,
            orientation='h',
            traceorder="normal",
            itemsizing='trace'),

        plot_bgcolor="White",
        xaxis_showticklabels=False
    )

    # Add a horizontal white bar (gap) between the graphs and other decorative elements
    fig.update_layout(
        shapes=[
            # Background rectangles for visual segmentation
            {'type': 'rect', 'xref': 'paper', 'yref': 'paper', 'x0': 0, 'y0': 0, 'x1': 0.5, 'y1': 1, 'fillcolor': '#fccacc', 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'paper', 'x0': 0.5, 'y0': 0, 'x1': 1, 'y1': 1, 'fillcolor': "#ced4f5", 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'y', 'x0': -0.1, 'y0': 6.7, 'x1': 1.1, 'y1': 7.4, 'fillcolor': 'white', 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'y', 'x0': -0.1, 'y0': 10.7, 'x1': 1.1, 'y1': 11.4, 'fillcolor': 'white', 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'y', 'x0': -0.1, 'y0': 23.5, 'x1': 1.1, 'y1': 24.2, 'fillcolor': 'white', 'layer': 'below', 'line': {'width': 0}},

            # Decorative lines
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 24.2, 'y1': 24.2, 'xref': "paper", 'line': {'color': "gray", 'width': 2}},
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 11.4, 'y1': 11.4, 'xref': "paper", 'line': {'color': "gray", 'width': 2}},
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 7.4, 'y1': 7.4, 'xref': "paper", 'line': {'color': "gray", 'width': 2}},
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 1.02, 'y1': 1.02, 'xref': "paper", 'yref': "paper", 'line': {'color': "gray", 'width': 2}}
        ],
    )

    # Add a horizontal white bar (gap) between the graphs and other decorative elements
    fig.update_layout(
        shapes=[
            # Background rectangles for visual segmentation
            {'type': 'rect', 'xref': 'paper', 'yref': 'paper', 'x0': 0, 'y0': 0, 'x1': 0.5, 'y1': 1, 'fillcolor': '#fccacc', 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'paper', 'x0': 0.5, 'y0': 0, 'x1': 1, 'y1': 1, 'fillcolor': "#ced4f5", 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'y', 'x0': -0.1, 'y0': 6.7, 'x1': 1.1, 'y1': 7.4, 'fillcolor': 'white', 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'y', 'x0': -0.1, 'y0': 10.7, 'x1': 1.1, 'y1': 11.4, 'fillcolor': 'white', 'layer': 'below', 'line': {'width': 0}},
            {'type': 'rect', 'xref': 'paper', 'yref': 'y', 'x0': -0.1, 'y0': 23.5, 'x1': 1.1, 'y1': 24.2, 'fillcolor': 'white', 'layer': 'below', 'line': {'width': 0}},

            # Decorative lines
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 24.2, 'y1': 24.2, 'xref': "paper", 'line': {'color': "gray", 'width': 2}},
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 11.4, 'y1': 11.4, 'xref': "paper", 'line': {'color': "gray", 'width': 2}},
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 7.4, 'y1': 7.4, 'xref': "paper", 'line': {'color': "gray", 'width': 2}},
            {'type': "line", 'x0': -1.6, 'x1': 1, 'y0': 1.02, 'y1': 1.02, 'xref': "paper", 'yref': "paper", 'line': {'color': "gray", 'width': 2}}
        ],
    )




    fig.add_annotation(x=-1.6, y=30, text='<b>External</b>', showarrow=False, arrowhead=1,  xref='paper')
    fig.add_annotation(x=-1.6, y=23.8, text='<b>Internal</b>', showarrow=False, arrowhead=1, xref='paper')
    fig.add_annotation(x=-1.6, y=10.9, text='<b>MAT</b>', showarrow=False, arrowhead=1,xref='paper')
    fig.add_annotation(x=-1.6, y=6.9, text='<b>Patients</b>', showarrow=False, arrowhead=1, xref='paper')
    
    fig.add_annotation(x=0.05, y=1.05, text='<b>Barriers</b>', showarrow=False, arrowhead=1,xref='paper',yref='paper')
    fig.add_annotation(x=0.5, y=1.05, text='<b>Neutral</b>', showarrow=False, arrowhead=1,xref='paper',yref='paper')
    fig.add_annotation(x=1, y=1.05, text='<b>Facilitators</b>', showarrow=False, arrowhead=1,xref='paper',yref='paper')


    fig.update_traces( marker=dict(size=40,line=dict(color='#535454', width=2)))    
    legend_image_path = 'legend_onlybaseline.png' 
    legend_image = dict(source=encode_image(legend_image_path),xref="paper",yref="paper",x=-0.2,y=-0.04,sizex=1.8,sizey=1,xanchor="center",yanchor="middle",)

    # Add the image to the layout
    fig.add_layout_image(legend_image)


    fig.update_layout(
        height=2000,
        # width = 2667,
        width=1800,
        title_font=dict(size=40),font=dict(size=38),
        margin=dict(l=1000, r=50, t=150, b=150, pad = 15),
        )
    
    fig.update_xaxes(range=[0.5, 5.5])    

    # fig.write_image("images_out/SITT-MATT_sites/dummy/dummy_IFASIS_graph.png")

    fig.write_image("images_out/SITT-MATT_sites/feb_2024/" + df['site_name'].loc[0] + "_IFASIS_graph.png")
    # fig.write_image("images_out/SITT-MATT_sites/feb_2024/"+ df['site_name'].loc[0] + "_IFASIS_graph.svg")

In [ ]:
# plot_df(df_dummy)

In [ ]:
for df in dfs:
    if df['site_name'].iloc[0] == 'id04':
        plot_df(df)

In [ ]:
len(dfs)

In [ ]:
# !cp -r "images_out/SITT-MATT_sites/feb_2024" "/Users/liachin-purcell/Library/CloudStorage/Box-Box/McGovern Lab/MAT for OUD/SITT-MAT/03. EMF/February 2024/IFASIS"